In [1]:
import sys,os
import numpy as np
import pandas as pd

from utils.method import read_bic_table

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read sample annotations 
cohort = "BRCA"
annot = pd.read_csv("data/PanCan/"+cohort+".annotation.tsv.gz",sep ="\t",index_col=0)

#annot["stage_3+4"] = annot["stage_3"] + annot["stage_4"] 
print("total samples:",annot.shape[0])
annot.loc[:,["stage_2","stage_3","stage_4","age","sex"]].sum()

total samples: 1098


stage_2      622.0
stage_3      250.0
stage_4       20.0
age        64019.0
sex         1086.0
dtype: float64

In [3]:
# read biclusters 
fname = cohort+".consensus_kmeans.42.bin=kmeans,pval=0.01,clust=WGCNA,ds=0,dch=0.995.biclsuters.tsv"
folder = "/home/olya/Downloads/"+cohort+"/"

biclusters = read_bic_table(folder+fname)
print("All biclusters:", biclusters.shape[0])
biclusters = biclusters.loc[biclusters["detected_n_times"]>1,:]
print("Detected 2+ times:", biclusters.shape[0])

All biclusters: 149
Detected 2+ times: 110


# Survival analysis

In [4]:
from utils.eval import  add_survival

covariates =["age","sex","stage_2","stage_3","stage_4"]

non_na = annot.loc[:,["stage_2","stage_3","stage_4","age","sex"]].count()
non_zero = annot.loc[:,["stage_2","stage_3","stage_4","age","sex"]]
non_zero = non_zero[non_zero>0].count()+non_zero[non_zero<0].count() # bc of NA

# exclude covariates with too many missing values <10 non-NA
# exclude covariates defining too small groups 5<non-zero or ones > N-5
covariates_to_remove = []
for c in covariates:
    if non_na[c]<10:
        covariates_to_remove.append(c)
        print("\tremoved",c,"from covariates (too many NA)",file=sys.stderr)
    elif non_zero[c]<5 or annot[c][annot[c]==0].count()>annot.shape[0]-5:
        covariates_to_remove.append(c)
        print("\tremoved",c,"from covariates (too small group)",file=sys.stderr)
covariates = [c for c in covariates if not c in covariates_to_remove]

print("Covariates added to the model:", covariates)

Covariates added to the model: ['age', 'sex', 'stage_2', 'stage_3', 'stage_4']


In [5]:
# add survival analysis results to bicluster table
biclusters = add_survival(biclusters, annot, event= "OS",
                          surv_time ="OS.time",covariates = covariates)
biclusters = add_survival(biclusters, annot, event= "DFI",
                          surv_time ="DFI.time",covariates = covariates)
biclusters = add_survival(biclusters, annot, event= "PFI",
                          surv_time ="PFI.time",covariates = covariates)

biclusters

perfect separation for biclsuter of  12/1061 samples variances: 0.00 0.01
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1575: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.211. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(
stage_4 with zero variance excluded
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation match

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column sex have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['DFI'].astype(bool)
>>> print(df.loc[events, 'sex'].var())
>>> print(df.loc[~events, 'sex'].var())

A very low variance means that the column sex completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Co

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column sex have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['DFI'].astype(bool)
>>> print(df.loc[events, 'sex'].var())
>>> print(df.loc[~events, 'sex'].var())

A very low variance means that the column sex completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Co

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column sex have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['DFI'].astype(bool)
>>> print(df.loc[events, 'sex'].var())
>>> print(df.loc[~events, 'sex'].var())

A very low variance means that the column sex completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Co

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column sex have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['DFI'].astype(bool)
>>> print(df.loc[events, 'sex'].var())
>>> print(df.loc[~events, 'sex'].var())

A very low variance means that the column sex completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Co

...

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1669: RuntimeWarning: invalid value encountered in add
  risk_phi_x_x = risk_phi_x_x + phi_x_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encou

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1575: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.211. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).differenc

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1699: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1695: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1697

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:

,SNR,n_genes,n_samples,genes,samples,genes_up,genes_down,gene_indexes,sample_indexes,direction,...,DFI.LogR_p_value,DFI.p_value_BH,DFI.LogR_p_value_BH,PFI.p_value,PFI.HR,PFI.upper_95CI,PFI.lower_95CI,PFI.LogR_p_value,PFI.p_value_BH,PFI.LogR_p_value_BH
0,6.603591,2067,13,"{RP3-472M2.2, RP11-333B6.1, RP11-90E5.1, LATS2...","{TCGA-AC-A2QH-01, TCGA-A7-A26F-01, TCGA-E9-A24...","{RP3-472M2.2, RP11-333B6.1, RP11-90E5.1, LATS2...",{},"{24580, 24581, 32780, 8208, 24593, 24592, 2460...","{448, 810, 1035, 460, 45, 442, 338, 981, 917, ...",UP,...,0.526908,0.726687,0.789994,NaN,NaN,NaN,NaN,0.275693,NaN,0.583198
1,4.498101,7,7,"{LCE1A, LCE2C, LINC00302, LCE2D, LCE1F, LCE2B,...","{TCGA-E9-A245-01, TCGA-E9-A295-01, TCGA-AC-A2Q...","{LCE1A, LCE2C, LINC00302, LCE2D, LCE1F, LCE2B,...",{},"{15168, 15169, 15170, 15410, 15161, 15164, 15166}","{931, 518, 810, 338, 889, 634, 188}",UP,...,0.479807,0.603781,0.789994,NaN,NaN,NaN,NaN,0.757559,NaN,0.929303
2,4.037331,2,122,"{ACTG1P11, ACTG1P2}","{TCGA-AC-A2QH-01, TCGA-A8-A07U-01, TCGA-A2-A0E...","{ACTG1P11, ACTG1P2}",{},"{1897, 1905}","{514, 515, 4, 7, 1032, 522, 12, 1039, 16, 1045...",UP,...,0.083833,0.358018,0.431497,NaN,NaN,NaN,NaN,0.036803,NaN,0.238135
6,2.816966,4,13,"{TSPY10, TSPY8, TSPY3, TSPY2}","{TCGA-WT-AB41-01, TCGA-OL-A6VR-01, TCGA-AO-A1K...","{TSPY10, TSPY8, TSPY3, TSPY2}",{},"{37048, 37044, 37052, 37046}","{32, 416, 451, 548, 1033, 842, 555, 207, 752, ...",UP,...,0.335689,NaN,0.659389,NaN,NaN,NaN,NaN,0.242414,NaN,0.554822
7,2.766905,5,75,"{PRR20B, PRR20C, PRR20E, PRR20A, PRR20D}","{TCGA-A2-A4S3-01, TCGA-EW-A1PD-01, TCGA-LD-A7W...","{PRR20B, PRR20C, PRR20E, PRR20A, PRR20D}",{},"{21382, 21383, 21384, 21385, 21386}","{128, 512, 514, 2, 260, 132, 901, 1027, 387, 1...",UP,...,0.919120,0.914828,0.960947,NaN,NaN,NaN,NaN,0.880101,NaN,0.968112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,1.330680,219,364,"{SRCAP, CTD-2325A15.5, AC006014.7, TVP23C-CDRT...","{TCGA-BH-A0EE-01, TCGA-AR-A5QP-01, TCGA-OL-A66...",{},"{SRCAP, CTD-2325A15.5, AC006014.7, TVP23C-CDRT...","{3077, 22550, 8215, 8216, 25626, 13338, 2588, ...","{0, 1, 2, 3, 5, 7, 8, 11, 14, 16, 18, 28, 30, ...",DOWN,...,0.150813,0.366120,0.488284,NaN,NaN,NaN,NaN,0.896614,NaN,0.971126
141,1.314882,8,450,"{RP11-313J2.1, ZNF471, ZNF229, ZNF285, ZNF300,...","{TCGA-A8-A09X-01, TCGA-BH-A0EE-01, TCGA-OL-A66...",{},"{RP11-313J2.1, ZNF471, ZNF229, ZNF285, ZNF300,...","{38950, 38759, 38802, 25906, 38930, 39095, 390...","{0, 1, 6, 9, 13, 14, 15, 17, 23, 28, 30, 32, 3...",DOWN,...,0.941224,0.905554,0.960947,0.164232,0.777362,1.108514,0.545137,0.560535,0.211661,0.838812
142,1.308662,15,286,"{ELAVL3, NAT16, CELSR3, OGDHL, RIMS2, TMEM151A...","{TCGA-AC-A2QH-01, TCGA-3C-AALI-01, TCGA-E2-A15...","{ELAVL3, NAT16, CELSR3, OGDHL, RIMS2, TMEM151A...",{},"{4163, 24740, 21029, 19078, 24743, 6632, 5607,...","{0, 1018, 515, 517, 519, 8, 9, 10, 11, 524, 10...",UP,...,0.596087,0.753118,0.789994,NaN,NaN,NaN,NaN,0.625537,NaN,0.882168
145,1.225185,4,470,"{CTD-2554C21.3, ZNF90, ZNF626, CTD-2554C21.2}","{TCGA-A8-A09X-01, TCGA-A2-A0EQ-01, TCGA-AN-A03...",{},"{CTD-2554C21.3, ZNF90, ZNF626, CTD-2554C21.2}","{39276, 39067, 8091, 8092}","{0, 4, 5, 6, 11, 14, 17, 28, 31, 36, 38, 39, 4...",DOWN,...,0.257831,0.540924,0.587612,NaN,NaN,NaN,NaN,0.029833,NaN,0.238135


In [7]:
# adjusted p-value for OS, DFI or PFI is < 0.05:
df = biclusters[biclusters[["OS.p_value_BH","DFI.p_value_BH","PFI.p_value_BH"]].min(axis=1)<0.05]
df.loc[:,["SNR","n_genes","n_samples","genes","direction","OS.p_value_BH","DFI.p_value_BH","PFI.p_value_BH"]].sort_values(by="OS.p_value_BH")

,SNR,n_genes,n_samples,genes,direction,OS.p_value_BH,DFI.p_value_BH,PFI.p_value_BH
47,1.785266,327,364,"{EMX1, MIR2052HG, MS4A8, ARMT1, TCEA1P3, RP11-...",DOWN,0.004295,0.223533,NaN
54,1.677363,100,164,"{EPO, ISX, MGC32805, TBX10, FGG, ERBB2, RP11-7...",UP,0.029405,0.809255,0.097821
61,1.648707,267,420,"{RP4-616B8.6, MAD2L1, RP5-882O7.4, C8orf76, HG...",UP,0.029405,0.442107,0.097821
50,1.761369,2,477,"{AC005538.3, MSL3P1}",DOWN,0.042381,0.520138,NaN
96,1.511972,19,499,"{KCNK1, CDCA5, RP4-616B8.5, PLCH1, HMGB3, MYBL...",DOWN,0.042381,0.358018,NaN
118,1.447932,4,414,"{RP11-44F14.10, RP11-44F14.8, RP11-44F14.2, RP...",DOWN,0.200795,0.210247,0.037297
120,1.435937,39,485,"{RP1-288H2.2, LIN7A, SYT1, RP11-56A10.1, VSTM2...",UP,0.258040,0.184871,0.037297
10,2.595844,2,467,"{RP4-765C7.2, RP11-343H5.4}",UP,0.260245,0.452171,0.049151


In [12]:
# log-rank test
df = biclusters[biclusters[["OS.LogR_p_value_BH","DFI.LogR_p_value_BH","PFI.LogR_p_value_BH"]].min(axis=1)<0.05]
df.loc[:,["SNR","n_genes","n_samples","genes","direction","OS.LogR_p_value_BH","DFI.LogR_p_value_BH","PFI.LogR_p_value_BH"]].sort_values(by="OS.LogR_p_value_BH")

,SNR,n_genes,n_samples,genes,direction,OS.LogR_p_value_BH,DFI.LogR_p_value_BH,PFI.LogR_p_value_BH


In [16]:
" ".join(sorted(biclusters.loc[120,"genes"]))


'ATRNL1 CH507-236L23.3 CH507-338C24.3 CH507-39O4.3 CITF22-24E5.1 EDDM3B ENPP7P11 FAM153C FAM3B IGDCC3 IL19 LGI1 LIN7A LINC00052 LLNLF-158E9.1 MS4A8 NUTM2F PVALB RP1 RP1-288H2.2 RP11-284J1.1 RP11-328P23.3 RP11-462G12.2 RP11-479O9.4 RP11-497G19.1 RP11-532F6.4 RP11-532F6.5 RP11-56A10.1 RP11-791G16.2 RP11-96D1.6 RP4-735C1.4 RPL7AP26 SALL4P7 SALRNA1 SOCS2-AS1 SYT1 TMEM150C VSTM2A VSTM2A-OT1'

### Associaiton with sex

In [17]:
# male samples
males = set(annot.loc[annot["sex"]==0,:].index.values) 
females = set(annot.loc[annot["sex"]==1,:].index.values) 
# all samples
N = annot.shape[0]  
print("%s males, %s females"%(len(males), len(females)))

12 males, 1086 females


In [18]:
from utils.eval import add_sex

add_sex(biclusters,males = males,females=females).sort_values(by=["sex.pval_BH"])

,SNR,n_genes,n_samples,genes,samples,genes_up,genes_down,gene_indexes,sample_indexes,direction,...,PFI.p_value_BH,PFI.LogR_p_value_BH,male.pval,male.counts,male.pval_BH,female.pval,female.counts,female.pval_BH,sex.pval_BH,sex
119,1.445399,40,38,"{CDY18P, TTTY14, HSFY2, XKRY, AC022486.1, BCOR...","{TCGA-AO-A0JC-01, TCGA-V7-A7HQ-01, TCGA-EW-A1P...","{CDY18P, TTTY14, HSFY2, XKRY, AC022486.1, BCOR...",{},"{39170, 37259, 11917, 23697, 786, 27157, 5655,...","{896, 387, 273, 658, 767, 915, 153, 923, 160, ...",UP,...,0.998613,0.338012,4.486112e-19,"(12, 26, 0, 1060)",4.934723e-17,1.000000,"(26, 12, 1060, 0)",1.0,4.934723e-17,male
31,2.075729,14,63,"{RP11-69H14.6, RP11-983G14.1, RP11-2F9.4, OR4H...","{TCGA-A2-A4S3-01, TCGA-AC-A2QH-01, TCGA-OL-A97...","{RP11-69H14.6, RP11-983G14.1, RP11-2F9.4, OR4H...",{},"{15874, 19363, 19362, 19365, 30349, 30350, 184...","{387, 390, 777, 522, 141, 1039, 144, 18, 786, ...",UP,...,NaN,0.929303,3.079526e-04,"(5, 58, 7, 1028)",1.693739e-02,0.999980,"(58, 5, 1028, 7)",1.0,1.693739e-02,male
102,1.497493,75,487,"{ZC4H2, MOGAT2, LTF, MET, VWDE, CNTNAP3B, ZNF2...","{TCGA-A8-A09X-01, TCGA-BH-A0EE-01, TCGA-A2-A0E...",{},"{ZC4H2, MOGAT2, LTF, MET, VWDE, CNTNAP3B, ZNF2...","{27393, 35717, 27014, 6150, 17160, 39049, 9354...","{0, 4, 5, 6, 7, 8, 10, 13, 15, 23, 29, 30, 32,...",DOWN,...,NaN,0.761532,8.808928e-04,"(11, 476, 1, 610)",2.422455e-02,0.999946,"(476, 11, 610, 1)",1.0,2.422455e-02,male
7,2.766905,5,75,"{PRR20B, PRR20C, PRR20E, PRR20A, PRR20D}","{TCGA-A2-A4S3-01, TCGA-EW-A1PD-01, TCGA-LD-A7W...","{PRR20B, PRR20C, PRR20E, PRR20A, PRR20D}",{},"{21382, 21383, 21384, 21385, 21386}","{128, 512, 514, 2, 260, 132, 901, 1027, 387, 1...",UP,...,NaN,0.968112,7.072819e-04,"(5, 70, 7, 1016)",2.422455e-02,0.999944,"(70, 5, 1016, 7)",1.0,2.422455e-02,male
101,1.505756,15,525,"{IGLV7-43, IGKV1D-33, CD79A, IGKV3D-20, IGKV2D...","{TCGA-BH-A0EE-01, TCGA-AN-A03Y-01, TCGA-A2-A0Y...",{},"{IGLV7-43, IGKV1D-33, CD79A, IGKV3D-20, IGKV2D...","{5473, 13729, 13636, 36422, 13543, 13673, 6254...","{0, 5, 6, 7, 8, 10, 14, 15, 18, 19, 23, 25, 29...",DOWN,...,NaN,0.191874,1.921151e-03,"(11, 514, 1, 572)",4.226533e-02,0.999866,"(514, 11, 572, 1)",1.0,4.226533e-02,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,1.761369,2,477,"{AC005538.3, MSL3P1}","{TCGA-A8-A09X-01, TCGA-A1-A0SD-01, TCGA-OL-A66...",{},"{AC005538.3, MSL3P1}","{17632, 400}","{0, 3, 5, 6, 7, 11, 12, 17, 19, 21, 22, 27, 31...",DOWN,...,NaN,0.342555,6.568946e-01,"(5, 472, 7, 614)",1.000000e+00,0.571169,"(472, 5, 614, 7)",1.0,1.000000e+00,
46,1.807128,46,51,"{RP11-790I12.6, SPINT3, SULT1C3, UGT2B28, HMGB...","{TCGA-A8-A09X-01, TCGA-LL-A73Z-01, TCGA-C8-A27...","{RP11-790I12.6, SPINT3, SULT1C3, UGT2B28, HMGB...",{},"{4354, 29699, 10629, 28550, 37515, 37516, 5388...","{4, 265, 523, 12, 1036, 14, 141, 532, 660, 155...",UP,...,NaN,0.671802,1.000000e+00,"(0, 51, 12, 1035)",1.000000e+00,0.563445,"(51, 0, 1035, 12)",1.0,1.000000e+00,
43,1.842696,131,286,"{CTSLP2, RP11-40C6.2, CH507-42P11.7, RP11-234G...","{TCGA-AC-A2QH-01, TCGA-3C-AALI-01, TCGA-E9-A3H...","{CTSLP2, RP11-40C6.2, CH507-42P11.7, RP11-234G...",{},"{25091, 9219, 11270, 20998, 21011, 34326, 2306...","{513, 2, 1027, 1028, 515, 5, 1, 509, 9, 1033, ...",UP,...,NaN,0.761445,6.422413e-01,"(3, 283, 9, 803)",1.000000e+00,0.616343,"(283, 3, 803, 9)",1.0,1.000000e+00,
54,1.677363,100,164,"{EPO, ISX, MGC32805, TBX10, FGG, ERBB2, RP11-7...","{TCGA-3C-AALI-01, TCGA-AR-A2LH-01, TCGA-A8-A09...","{EPO, ISX, MGC32805, TBX10, FGG, ERBB2, RP11-7...",{},"{28160, 15884, 26637, 30738, 11810, 6180, 6181...","{513, 4, 6, 520, 1034, 11, 12, 1036, 14, 527, ...",UP,...,0.097821,0.238135,8.579911e-01,"(1, 163, 11, 923)",1.000000e+00,0.444797,"(163, 1, 923, 11)",1.0,1.000000e+00,
